In [1]:
#!/usr/bin/env python3
"""
Three-Body Problem Simulation with Chaos Visualization
"""

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib.collections import LineCollection
import argparse
import sys
import os
from tqdm import tqdm

def compute_acceleration(positions, masses, G=1.0, softening=0.1):
    """
    Compute acceleration for three-body problem
    
    Args:
        positions: (3, 2) array of body positions
        masses: (3,) array of masses
        G: gravitational constant
        softening: softening parameter
        
    Returns:
        accelerations: (3, 2) array of accelerations
    """
    n = positions.shape[0]
    accelerations = np.zeros((n, 2))
    
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
                
            dx = positions[j, 0] - positions[i, 0]
            dy = positions[j, 1] - positions[i, 1]
            r_squared = dx**2 + dy**2 + softening**2
            r = np.sqrt(r_squared)
            factor = G * masses[j] / (r_squared * r)
            
            accelerations[i, 0] += dx * factor
            accelerations[i, 1] += dy * factor
            
    return accelerations

def main():
    # Set up simulation parameters for chaotic three-body problem
    args = argparse.Namespace()
    args.n_bodies = 3  # Three-body problem
    args.t_end = 30.0  # Longer simulation time to show chaos
    args.dt = 0.001    # Smaller time step for accuracy
    args.output = 'three_body_chaos.mp4'
    args.size = 2.0    # Smaller area to keep bodies close
    args.mass_min = 0.8
    args.mass_max = 1.2
    args.softening = 0.05
    args.fps = 30
    args.trail = 100   # Longer trails to show chaotic paths
    args.dpi = 100
    args.colors = 'red,green,blue'  # Distinct colors for each body
    args.show_full_trajectory = True
    args.trail_alpha = 0.7
    args.trail_fade = True
    args.video_quality = 8
    args.adaptive_step = True  # Use adaptive stepping for accuracy

    # Initialize three bodies in a close configuration
    np.random.seed(42)  # For reproducibility
    masses = np.array([1.0, 1.0, 1.0])  # Equal masses for classic three-body problem
    
    # Initial positions - close to each other but not perfectly symmetric
    positions = np.array([
        [0.0, 0.0],     # Body 1 at center
        [0.5, 0.0],     # Body 2 slightly to the right
        [0.25, 0.433]   # Body 3 forming an equilateral triangle (but not quite)
    ])
    
    # Initial velocities - small perturbations to create chaos
    velocities = np.array([
        [0.0, 0.0],        # Body 1 initially at rest
        [0.0, 0.5],         # Body 2 moving upward
        [-0.5, 0.0]         # Body 3 moving left
    ]) * 0.7

    # Generate body colors
    body_colors = ['red', 'green', 'blue']

    # Initialize acceleration
    accelerations = compute_acceleration(positions, masses, softening=args.softening)
    
    # Pre-calculate number of frames
    n_frames = int(args.t_end / args.dt)
    store_every = max(1, int(n_frames / 1000))  # Store max 1000 frames
    stored_frames = []
    stored_positions = []
    
    # Velocity Verlet integration with adaptive time stepping
    print("\nRunning three-body simulation...")
    current_time = 0.0
    frame_count = 0
    stored_frames.append(frame_count)
    stored_positions.append(positions.copy())
    
    with tqdm(total=args.t_end) as pbar:
        while current_time < args.t_end:
            # Adaptive time stepping
            dt = args.dt
            if args.adaptive_step:
                # Calculate characteristic time scales
                acc_magnitudes = np.sqrt(accelerations[:,0]**2 + accelerations[:,1]**2)
                vel_magnitudes = np.sqrt(velocities[:,0]**2 + velocities[:,1]**2)
                
                max_acc = np.max(acc_magnitudes)
                max_vel = np.max(vel_magnitudes)
                
                if max_acc > 1e-10 and max_vel > 1e-10:
                    dt_acc = 0.1 * np.sqrt(1.0 / max_acc)
                    dt_vel = 0.1 * 1.0 / max_vel
                    dt = min(dt_acc, dt_vel, args.dt*2)
                    dt = max(dt, args.dt/10)
            
            # Update positions
            positions += velocities * dt + 0.5 * accelerations * dt**2
            
            # Update acceleration
            new_accelerations = compute_acceleration(positions, masses, softening=args.softening)
            
            # Update velocities
            velocities += 0.5 * (accelerations + new_accelerations) * dt
            accelerations = new_accelerations
            
            current_time += dt
            frame_count += 1
            
            # Store trajectory periodically
            if frame_count % store_every == 0 or current_time >= args.t_end:
                stored_frames.append(frame_count)
                stored_positions.append(positions.copy())
            
            pbar.update(dt)
    
    # Convert stored positions to array
    trajectories = np.array(stored_positions)
    print(f"\nSimulation completed in {frame_count} steps. Stored {len(stored_frames)} frames.")
    
    # Setup plot
    print("Preparing visualization...")
    fig, ax = plt.subplots(figsize=(10, 10), dpi=args.dpi)
    ax.set_xlim(-args.size, args.size)
    ax.set_ylim(-args.size, args.size)
    ax.set_aspect('equal')
    ax.set_title('Three-Body Problem with Chaotic Motion')
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.grid(True, linestyle='--', alpha=0.3)
    
    # Create scatter plot for bodies
    scatter = ax.scatter(
        trajectories[0, :, 0],
        trajectories[0, :, 1],
        s=100,  # Larger dots for visibility
        c=body_colors,
        zorder=10
    )
    
    # Create trails
    trails = []
    for i in range(args.n_bodies):
        rgba = mcolors.to_rgba(body_colors[i], alpha=args.trail_alpha)
        trail, = ax.plot([], [], '-', lw=2.0, color=rgba, zorder=5)
        trails.append(trail)
    
    # Full trajectory visualization
    full_trajectories = []
    time_cmap = plt.get_cmap('viridis')
    for i in range(args.n_bodies):
        segments = np.zeros((0, 2, 2))
        lc = LineCollection(segments, cmap=time_cmap, alpha=args.trail_alpha, linewidth=1.5, zorder=1)
        lc.set_array(np.array([]))
        full_trajectories.append(ax.add_collection(lc))
    
    # Animation update function
    def update(frame_idx):
        frame = stored_frames[frame_idx]
        pos = trajectories[frame_idx]
        
        # Update body positions
        scatter.set_offsets(pos)
        
        # Update trails
        if args.trail > 0:
            start_idx = max(0, frame_idx - args.trail)
            for i, trail in enumerate(trails):
                x = trajectories[start_idx:frame_idx+1, i, 0]
                y = trajectories[start_idx:frame_idx+1, i, 1]
                trail.set_data(x, y)
        
        # Update full trajectories
        if frame_idx > 0:
            for i in range(args.n_bodies):
                points = trajectories[:frame_idx+1, i, :]
                segments = np.array([points[:-1], points[1:]]).transpose(1, 0, 2)
                times = np.linspace(0, 1, len(segments))
                
                full_trajectories[i].set_segments(segments)
                full_trajectories[i].set_array(times)
                
                if args.trail_fade:
                    alpha_values = np.linspace(0.1, args.trail_alpha, len(segments))
                    full_trajectories[i].set_alpha(alpha_values)
        
        return [scatter] + trails + full_trajectories
    
    # Create animation
    n_frames = len(stored_frames)
    ani = FuncAnimation(
        fig, 
        update, 
        frames=n_frames,
        interval=1000/args.fps,
        blit=True
    )
    
    # Save animation
    try:
        writer = FFMpegWriter(
            fps=args.fps,
            metadata={'artist': 'Three-Body Simulation'},
            bitrate=1800,
            codec='libx264',
        )

        print(f"\nRendering video...")
        with tqdm(total=n_frames, desc="Rendering", unit="frame") as pbar:
            ani.save(
                args.output,
                writer=writer,
                dpi=args.dpi,
                progress_callback=lambda i, n: pbar.update(1)
            )
        
        print(f"Video saved to: {os.path.abspath(args.output)}")
    except RuntimeError as e:
        print(f"\nERROR saving video: {e}")
        sys.exit(1)
    
    plt.close(fig)

if __name__ == "__main__":
    main()


Running three-body simulation...


 92%|█████████▏| 27.575999999995595/30.0 [00:00<00:00, 30.75it/s]/usr/local/lib/python3.11/dist-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 30.00199999999425/30.0 [00:00<00:00, 30.42it/s] 



Simulation completed in 15001 steps. Stored 502 frames.
Preparing visualization...

Rendering video...


Rendering: 100%|██████████| 502/502 [01:16<00:00,  6.59frame/s]

Video saved to: /kaggle/working/three_body_chaos.mp4
